# IMPORTS

In [1]:
#!pip install ccxt
#!pip install xlrd

import ccxt
import pandas as pd
import time
from datetime import datetime
import myconfig
import math
import xlrd

# INITIALIZE

In [2]:
exchange = ccxt.binance({
    "apiKey": myconfig.API_KEY,
    "secret": myconfig.API_SECRET
})

#exchange = ccxt.binance ({
     
#    'apiKey': myconfig.API_KEY,
#    'secret': myconfig.API_SECRET,
#    'rateLimit': 10000,  # unified exchange property
#    'options': {
#        'adjustForTimeDifference': True,  # exchange-specific option
#   }
#})
#exchange.options['adjustForTimeDifference'] = False

In [3]:
# markets = exchange.fetchMarkets()
# market_symbols = [market['symbol'] for market in markets]
# print(f'No. of market symbols: {len(market_symbols)}')
# print(f'Sample:{market_symbols[0:5]}')

In [4]:
path = ('.\mybuy.xls')
trade_combinations = pd.read_excel(path)
trade_combinations

,Type,sym1,sym2,sym3
0,BUY_BUY_SELL,POLS/BUSD,POLS/BNB,BNB/BUSD
1,BUY_BUY_SELL,SCRT/USDT,SCRT/ETH,ETH/BUSD
2,BUY_BUY_SELL,SAND/USDT,SAND/ETH,ETH/USDT
3,BUY_BUY_SELL,ETH/USDT,ATOM/ETH,ATOM/USDT
4,BUY_BUY_SELL,ETH/USDT,NEAR/ETH,NEAR/USDT
...,...,...,...,...
561,BUY_BUY_SELL,BTC/USDT,OP/BTC,OP/USDT
562,BUY_SELL_SELL,OP/USDT,OP/BTC,BTC/USDT
563,BUY_BUY_SELL,BTC/USDT,STG/BTC,STG/USDT
564,BUY_SELL_SELL,STG/USDT,STG/BTC,BTC/USDT


In [5]:
trade_combinations.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 566 entries, 0 to 565
Data columns (total 4 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   Type    566 non-null    object
 1   sym1    566 non-null    object
 2   sym2    566 non-null    object
 3   sym3    566 non-null    object
dtypes: object(4)
memory usage: 17.8+ KB


# STEP 1: GET ALL THE CRYPTO COMBINATIONS FOR USDT

In [6]:
# def get_crypto_combinations(market_symbols, base):
#     combinations = []
#     for sym1 in market_symbols:
        
#         sym1_token1 = sym1.split('/')[0]
#         sym1_token2 = sym1.split('/')[1]
        
#         if (sym1_token2 == base):
#             for sym2 in market_symbols:
#                 sym2_token1 = sym2.split('/')[0]
#                 sym2_token2 = sym2.split('/')[1]
#                 if (sym1_token1 == sym2_token2):
#                     for sym3 in market_symbols:
#                         sym3_token1 = sym3.split('/')[0]
#                         sym3_token2 = sym3.split('/')[1]
#                         if((sym2_token1 == sym3_token1) and (sym3_token2 == sym1_token2)):
#                             combination = {
#                                 'base':sym1_token2,
#                                 'intermediate':sym1_token1,
#                                 'ticker':sym2_token1,
#                             }
#                             combinations.append(combination)
                

#     return combinations
        
# wx_combinations_usdt = get_crypto_combinations(market_symbols,'USDT')


In [7]:
# print(f'No. of crypto combinations: {len(wx_combinations_usdt)}')

# cominations_df = pd.DataFrame(wx_combinations_usdt)
# cominations_df.sample(10)

# STEP 2: PERFORM TRIANGULAR ARBITRAGE

## Utility method to fetch the current ticker price

In [8]:
def fetch_current_ticker_price(ticker):
    current_ticker_details = exchange.fetch_ticker(ticker)
    ticker_price = current_ticker_details['close'] if current_ticker_details is not None else None
    return ticker_price

In [9]:
def check_if_float_zero(value):
    return math.isclose(value, 0.0, abs_tol=1e-9)

In [10]:
def fetch_ticker_bid(ticker):
    current_ticker_details = exchange.fetch_ticker(ticker)
    ticker_price = current_ticker_details['bid'] if current_ticker_details is not None else None
    return ticker_price

In [11]:
def fetch_ticker_ask(ticker):
    current_ticker_details = exchange.fetch_ticker(ticker)
    ticker_price = current_ticker_details['ask'] if current_ticker_details is not None else None
    return ticker_price

In [12]:
def ticker_is_reasonable_percent(scrip, percentage):
    current_ask = fetch_ticker_ask(scrip)
    current_bid = fetch_ticker_bid(scrip)
    current_price = fetch_current_ticker_price(scrip)
    cond1 = current_ask is not None and not check_if_float_zero(current_ask)
    cond2 = current_bid is not None and not check_if_float_zero(current_bid)
    if cond1 and cond2:
        bid_ask_diff = (current_ask / current_bid - 1) * 100
        cond3 = bid_ask_diff <= percentage
    else:
        cond3 =False
        bid_ask_diff = None
    return (cond1 and cond2 and cond3, current_ask, current_price, current_bid, bid_ask_diff)

In [13]:
def ticker_is_reasonable(scrip):
    print('___________________________________________________________________________________')
    print('Scrip:', scrip)
    is_valid, ask_prev, current_price, bid_prev, bid_ask_diff = ticker_is_reasonable_percent(scrip, 0.14)
    if is_valid:
        print('bid:',bid_prev,'curr_price: ',current_price,'ask:',ask_prev,'bid_ask_diff:', bid_ask_diff,'%, \nEnter')
        print('___________________________________________________________________________________')
        return True
    is_valid, ask_prev, current_price, bid_prev, bid_ask_diff = ticker_is_reasonable_percent(scrip, 1)
    if is_valid:
        print('bid:',bid_prev,'curr_price: ',current_price,'ask:',ask_prev,'bid_ask_diff:', bid_ask_diff,'%, \nWait 3 sec')
        time.sleep(3)
        is_valid, ask_new, current_price, bid_new, bid_ask_diff = ticker_is_reasonable_percent(scrip, 1)
        print('Last_bid:',bid_prev,'New_bid:',bid_new,'\ncurr_price: ',current_price,'\nLast_ask:',ask_prev,'New_ask:',ask_prev,'\nbid_ask_diff:', bid_ask_diff,'%')
        if ask_new != ask_prev and bid_new != bid_prev:
            print('Last_bid:',bid_prev,'New_bid:',bid_new,'\ncurr_price: ',current_price,'\nLast_ask:',ask_prev,'New_ask:',ask_prev,'\nbid_ask_diff:', bid_ask_diff,'% \nEnter')
            print('___________________________________________________________________________________')
            return True
        print('Last_bid:',bid_prev,'New_bid:',bid_new,'\ncurr_price: ',current_price,'\nLast_ask:',ask_prev,'New_ask:',ask_prev,'\nbid_ask_diff:', bid_ask_diff,'% \nWait 3 sec')
        ask_prev, bid_prev = ask_new, bid_prev
        time.sleep(3)
        is_valid, ask_new, current_price, bid_new, bid_ask_diff = ticker_is_reasonable_percent(scrip, 1)
        if ask_new != ask_prev and bid_new != bid_prev:
            print('Last_bid:',bid_prev,'New_bid:',bid_new,'\ncurr_price: ',current_price,'\nLast_ask:',ask_prev,'New_ask:',ask_prev,'\nbid_ask_diff:', bid_ask_diff,'% \nEnter')
            print('___________________________________________________________________________________')
            return True
    print('Last_bid:',bid_prev,'\ncurr_price: ',current_price,'\nLast_ask:',ask_prev,'\nbid_ask_diff:', bid_ask_diff,'% \nSkip')
    print('___________________________________________________________________________________')
    return False          

## Triangular Arbitrage

### Check with new conditions

In [14]:
def check_buy_buy_sell(scrip1, scrip2, scrip3, initial_investment):
    
    ## SCRIP1
    investment_amount1 = initial_investment
    current_price1 = fetch_current_ticker_price(scrip1)
    final_price = 0
    scrip_prices = {}

    if ticker_is_reasonable(scrip1) and current_price1 is not None and not check_if_float_zero(current_price1):
        buy_quantity1 = round(investment_amount1 / current_price1, 8)
        
        # TRY WITHOUT SLEEP IF THE EXCHANGE DOES NOT THROW RATE LIMIT EXCEPTIONS
        time.sleep(1)
        ## SCRIP2
        investment_amount2 = buy_quantity1     
        current_price2 = fetch_current_ticker_price(scrip2)
        if ticker_is_reasonable(scrip2) and current_price2 is not None and not check_if_float_zero(current_price2):
            buy_quantity2 = round(investment_amount2 / current_price2, 8)
            
            # TRY WITHOUT SLEEP IF THE EXCHANGE DOES NOT THROW RATE LIMIT EXCEPTIONS
            time.sleep(1)
            ## SCRIP3
            investment_amount3 = buy_quantity2     
            current_price3 = fetch_current_ticker_price(scrip3)
            if ticker_is_reasonable(scrip3) and current_price3 is not None and not check_if_float_zero(current_price3):
                sell_quantity3 = buy_quantity2
                final_price = round(sell_quantity3 * current_price3,3)
                scrip_prices = {scrip1 : current_price1, scrip2 : current_price2, scrip3 : current_price3}
                
    return final_price, scrip_prices

### Check with old conditions

In [15]:
# def check_buy_buy_sell(scrip1, scrip2, scrip3, initial_investment):
    
#     ## SCRIP1
#     investment_amount1 = initial_investment
#     current_price1 = fetch_current_ticker_price(scrip1)
#     final_price = 0
#     scrip_prices = {}

#     if current_price1 is not None and not check_if_float_zero(current_price1):
#         buy_quantity1 = round(investment_amount1 / current_price1, 8)

#         # TRY WITHOUT SLEEP IF THE EXCHANGE DOES NOT THROW RATE LIMIT EXCEPTIONS
#         time.sleep(1)
#         # SCRIP2
#         investment_amount2 = buy_quantity1     
#         current_price2 = fetch_current_ticker_price(scrip2)
#         if current_price2 is not None and not check_if_float_zero(current_price2):
#             buy_quantity2 = round(investment_amount2 / current_price2, 8)

#             # TRY WITHOUT SLEEP IF THE EXCHANGE DOES NOT THROW RATE LIMIT EXCEPTIONS
#             time.sleep(1)
#             ## SCRIP3
#             investment_amount3 = buy_quantity2     
#             current_price3 = fetch_current_ticker_price(scrip3)
#             if current_price3 is not None and not check_if_float_zero(current_price3):
#                 sell_quantity3 = buy_quantity2
#                 final_price = round(sell_quantity3 * current_price3,3)
#                 scrip_prices = {scrip1 : current_price1, scrip2 : current_price2, scrip3 : current_price3}
                
#     return final_price, scrip_prices

### Check with new conditions

In [16]:
def check_buy_sell_sell(scrip1, scrip2, scrip3, initial_investment):
    
    ## SCRIP1
    investment_amount1 = initial_investment
    current_price1 = fetch_current_ticker_price(scrip1)
    final_price = 0
    scrip_prices = {}
    ticker1_cond = ticker_is_reasonable(scrip1)
    ticker2_cond = ticker_is_reasonable(scrip2)
    ticker3_cond = ticker_is_reasonable(scrip3)
    if ticker1_cond and current_price1 is not None and not check_if_float_zero(current_price1):
        buy_quantity1 = round(investment_amount1 / current_price1, 8)
        
        # TRY WITHOUT SLEEP IF THE EXCHANGE DOES NOT THROW RATE LIMIT EXCEPTIONS
        #time.sleep(1)
        ## SCRIP2
        investment_amount2 = buy_quantity1     
        current_price2 = fetch_current_ticker_price(scrip2)
        if ticker2_cond and current_price2 is not None and not check_if_float_zero(current_price2):
            sell_quantity2 = buy_quantity1
            sell_price2 = round(sell_quantity2 * current_price2,8)
            
            # TRY WITHOUT SLEEP IF THE EXCHANGE DOES NOT THROW RATE LIMIT EXCEPTIONS
            #time.sleep(1)
            ## SCRIP1
            investment_amount3 = sell_price2     
            current_price3 = fetch_current_ticker_price(scrip3)
            if ticker3_cond and current_price3 is not None and not check_if_float_zero(current_price3):
                sell_quantity3 = sell_price2
                final_price = round(sell_quantity3 * current_price3,8)
                scrip_prices = {scrip1 : current_price1, scrip2 : current_price2, scrip3 : current_price3}
    return final_price,scrip_prices


### Check with old conditions

In [17]:
# def check_buy_sell_sell(scrip1, scrip2, scrip3, initial_investment):
#     ## SCRIP1
#     investment_amount1 = initial_investment
#     current_price1 = fetch_current_ticker_price(scrip1)
#     crurent_ask1 = fetch_ticker_ask(scrip1)
#     current_bid1 = fetch_ticker_bid(scrip1)
#     bid_ask_diff = (current_ask1 / current_bid1 - 1) * 100 
#     final_price = 0
#     scrip_prices = {}
#     if current_price1 is not None and not check_if_float_zero(current_price1):
#         buy_quantity1 = round(investment_amount1 / current_price1, 8)
        
#         # TRY WITHOUT SLEEP IF THE EXCHANGE DOES NOT THROW RATE LIMIT EXCEPTIONS
#         time.sleep(1)
#         ## SCRIP2
#         investment_amount2 = buy_quantity1     
#         current_price2 = fetch_current_ticker_price(scrip2)
#         if current_price2 is not None and not check_if_float_zero(current_price2):
#             sell_quantity2 = buy_quantity1
#             sell_price2 = round(sell_quantity2 * current_price2,8)
            
#             # TRY WITHOUT SLEEP IF THE EXCHANGE DOES NOT THROW RATE LIMIT EXCEPTIONS
#             time.sleep(1)
#             ## SCRIP1
#             investment_amount3 = sell_price2     
#             current_price3 = fetch_current_ticker_price(scrip3)
#             if current_price3 is not None and not check_if_float_zero(current_price3):
#                 sell_quantity3 = sell_price2
#                 final_price = round(sell_quantity3 * current_price3,3)
#                 scrip_prices = {scrip1 : current_price1, scrip2 : current_price2, scrip3 : current_price3}
#     return final_price,scrip_prices


In [18]:
def check_profit_loss(total_price_after_sell,initial_investment,transaction_brokerage, min_profit):
    apprx_brokerage = transaction_brokerage * initial_investment/100 * 3
    min_profitable_price = initial_investment + apprx_brokerage + min_profit
    profit_loss = round(total_price_after_sell - min_profitable_price,3)
    return profit_loss

# STEP 3: PLACE THE TRADE ORDERS

In [19]:
def place_buy_order(scrip, quantity, limit):
    order = exchange.create_limit_buy_order(scrip, quantity, limit)
    return order

def place_sell_order(scrip, quantity, limit):
    order = exchange.create_limit_sell_order(scrip, quantity, limit)
    return order 

def place_trade_orders(type, scrip1, scrip2, scrip3, initial_amount, scrip_prices):
    final_amount = 0.0
    if type == 'BUY_BUY_SELL':
        s1_quantity = initial_amount/scrip_prices[scrip1]
        place_buy_order(scrip1, s1_quantity, scrip_prices[scrip1])
        
        s2_quantity = s1_quantity/scrip_prices[scrip2]
        place_buy_order(scrip2, s2_quantity, scrip_prices[scrip2])
        
        s3_quantity = s2_quantity
        place_sell_order(scrip3, s3_quantity, scrip_prices[scrip3])
        
    elif type == 'BUY_SELL_SELL':
        s1_quantity = initial_amount/scrip_prices[scrip1]
        place_buy_order(scrip1, s1_quantity, scrip_prices[scrip1])
        
        s2_quantity = s1_quantity
        place_sell_order(scrip2, s2_quantity, scrip_prices[scrip2])
        
        s3_quantity = s2_quantity * scrip_prices[scrip2]
        place_sell_order(scrip3, s3_quantity, scrip_prices[scrip3])
        
        
    return final_amount

Sample order from exchange immediately after execution:   
{'info': {'id': '2490462375', 'symbol': 'btcusdt', 'type': 'limit', 'side': 'buy', 'status': 'wait', 'price': '43201.0', 'origQty': '0.002314', 'executedQty': '0.0', 'createdTime': '1646302254000', 'updatedTime': '1646302254000'}, 'id': '2490462375', 'clientOrderId': None, 'timestamp': 1646302254000, 'datetime': '2022-03-03T10:10:54.000Z', 'lastTradeTimestamp': 1646302254000, 'status': 'open', 'symbol': 'BTC/USDT', 'type': 'limit', 'timeInForce': None, 'postOnly': None, 'side': 'buy', 'price': 43201.0, 'amount': None, 'filled': 0.0, 'remaining': None, 'cost': 0.0, 'fee': None, 'average': None, 'trades': [], 'fees': []}

# STEP 4: WRAPPING IT TOGETHER

In [20]:
def perform_triangular_arbitrage(scrip1, scrip2, scrip3, arbitrage_type,initial_investment, 
                               transaction_brokerage, min_profit):
    final_price = 0.0
    
    if(arbitrage_type == 'BUY_BUY_SELL'):
        # Check this combination for triangular arbitrage: scrip1 - BUY, scrip2 - BUY, scrip3 - SELL
        final_price, scrip_prices = check_buy_buy_sell(scrip1, scrip2, scrip3,initial_investment)
        
    elif(arbitrage_type == 'BUY_SELL_SELL'):
        # Check this combination for triangular arbitrage: scrip1 - BUY, scrip2 - SELL, scrip3 - SELL
        final_price, scrip_prices = check_buy_sell_sell(scrip1, scrip2, scrip3,initial_investment)
        
    profit_loss = check_profit_loss(final_price,initial_investment, transaction_brokerage, min_profit)
    
    print(
        f"*{datetime.now().strftime('%H:%M:%S')}\t{final_price}\t{scrip_prices}\t"\
              f"{arbitrage_type}\t{scrip1}\t{scrip2}\t{scrip3}\tProfit/Loss: {round(final_price-initial_investment,5)} ")
        
    if profit_loss>0:
        print(
            f"**PROFIT{datetime.now().strftime('%H:%M:%S')}\t{final_price}\t{scrip_prices}\t"\
              f"{arbitrage_type}\t{scrip1}\t{scrip2}\t{scrip3}\tProfit/Loss: {round(final_price-initial_investment,5)} ")
        
        # UNCOMMENT THIS LINE TO PLACE THE ORDERS
        # place_trade_orders(arbitrage_type, scrip1, scrip2, scrip3, initial_investment, scrip_prices)



### New loop

In [22]:
INVESTMENT_AMOUNT_DOLLARS = 100
MIN_PROFIT_DOLLARS = 0.5
BROKERAGE_PER_TRANSACTION_PERCENT = 0.075

# UNCOMMENT THE WHILE LOOP TO RUN THIS LOOP CONTINUOUSLY
# while(1):
for i in range(len(trade_combinations)):

    s1 = trade_combinations.sym1[i]
    s2 = trade_combinations.sym2[i]
    s3 = trade_combinations.sym3[i]
    trans_type = trade_combinations.Type[i]

    # Check triangular arbitrage for buy-buy-sell 
    if trans_type == 'BUY_BUY_SELL':
        perform_triangular_arbitrage(s1,s2,s3,trans_type,INVESTMENT_AMOUNT_DOLLARS,
                                BROKERAGE_PER_TRANSACTION_PERCENT, MIN_PROFIT_DOLLARS)
    else:
        perform_triangular_arbitrage(s3,s2,s1,trans_type,INVESTMENT_AMOUNT_DOLLARS,
                                BROKERAGE_PER_TRANSACTION_PERCENT, MIN_PROFIT_DOLLARS)
    # Sleep to avoid rate limit on api calls (RateLimitExceeded exception)
    time.sleep(1)        
    

___________________________________________________________________________________
Scrip: POLS/BUSD
bid: 0.474 curr_price:  0.474 ask: 0.476 bid_ask_diff: 0.42194092827003704 %, 
Wait 3 sec
Last_bid: 0.474 New_bid: 0.474 
curr_price:  0.474 
Last_ask: 0.476 New_ask: 0.476 
bid_ask_diff: 0.42194092827003704 %
Last_bid: 0.474 New_bid: 0.474 
curr_price:  0.474 
Last_ask: 0.476 New_ask: 0.476 
bid_ask_diff: 0.42194092827003704 % 
Wait 3 sec
Last_bid: 0.474 
curr_price:  0.474 
Last_ask: 0.476 
bid_ask_diff: 0.42194092827003704 % 
Skip
___________________________________________________________________________________
*14:01:11	0	{}	BUY_BUY_SELL	POLS/BUSD	POLS/BNB	BNB/BUSD	Profit/Loss: -100 
___________________________________________________________________________________
Scrip: SCRT/USDT
bid: 0.954 curr_price:  0.954 ask: 0.955 bid_ask_diff: 0.10482180293500676 %, 
Enter
___________________________________________________________________________________
________________________________

KeyboardInterrupt: 

### Old loop

In [ ]:
# INVESTMENT_AMOUNT_DOLLARS = 100
# MIN_PROFIT_DOLLARS = 0.5
# BROKERAGE_PER_TRANSACTION_PERCENT = 0.075


# # UNCOMMENT THE WHILE LOOP TO RUN THIS LOOP CONTINUOUSLY
# #while(1):
# for combination in wx_combinations_usdt:

#     base = combination['base']
#     intermediate = combination['intermediate']
#     ticker = combination['ticker']


#     s1 = f'{intermediate}/{base}'    # Eg: BTC/USDT
#     s2 = f'{ticker}/{intermediate}'  # Eg: ETH/BTC
#     s3 = f'{ticker}/{base}'          # Eg: ETH/USDT 
# # Вывести через табуляцию , начиная с * все, что есть + цену

#     # Check triangular arbitrage for buy-buy-sell 
#     perform_triangular_arbitrage(s1,s2,s3,'BUY_BUY_SELL',INVESTMENT_AMOUNT_DOLLARS,
#                               BROKERAGE_PER_TRANSACTION_PERCENT, MIN_PROFIT_DOLLARS)
#     # Sleep to avoid rate limit on api calls (RateLimitExceeded exception)
#     time.sleep(1) 
#     # Check triangular arbitrage for buy-sell-sell 
#     perform_triangular_arbitrage(s3,s2,s1,'BUY_SELL_SELL',INVESTMENT_AMOUNT_DOLLARS,
#                               BROKERAGE_PER_TRANSACTION_PERCENT, MIN_PROFIT_DOLLARS)
#     time.sleep(1)    
         
    